Swiss Pairs Simulation in Python

We start with a simple pure-Python model of a tournament. For now, we're not going to worry about how to simulate multiple runs, much less about performance. We just want to get a basic model working, and then we can refine it.

We need a few imports. The random module for generating random numbers is crucial.

In [10]:
import random
from collections import Counter

In [4]:
# Keep track of the scores - at the start of the tournament, everyone has 0 wins
scores = {p:0 for p in range(20)}
# Keep track of who has already played who
played = set()

We need to find opponents for each pair. To do that, we sort pairs by score (and randomly within pairs with the same score),
then for each pair in turn assign as opponent the next pair in the list that they have not already played.

In [53]:
def pair(scores, played):
    pairs = list(sorted(scores.keys()))
    def key(p):
        # Rank by score, and then randomly for pairs with the same score
        return scores[p] #, random.random()
    ranked = list(sorted(pairs, key=key))
    games = []
    print("Ranking:", ranked)
    while ranked:
        # Get each pair in turn
        p1 = ranked.pop(0)
        print("Matching", p1, end="... ")
        # Find an opponent - the first pair they haven't played yet
        for n, p2 in enumerate(ranked):
            game = (p1, p2)
            print(p2, end=", ")
            if game not in played:
                del ranked[n]
                games.append(game)
                played.add(game)
                print("Found", p2)
                break
        else:
            # Looks like p1 has played every remaining pair.
            # In theory, there may be backtracking solutions that fail less often.
            # Don't worry about this for now, as the official rules are probably better anyway.
            print()
            raise RuntimeError("Unable to find a game for pair {}".format(p1))
    
    return games

# Try this out - print an initial pairing, then reset the variables for later
print(pair({p:0 for p in range(20)}, set()))

Ranking: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Matching 0... 1, Found 1
Matching 2... 3, Found 3
Matching 4... 5, Found 5
Matching 6... 7, Found 7
Matching 8... 9, Found 9
Matching 10... 11, Found 11
Matching 12... 13, Found 13
Matching 14... 15, Found 15
Matching 16... 17, Found 17
Matching 18... 19, Found 19
[(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11), (12, 13), (14, 15), (16, 17), (18, 19)]


We need to simulate playing a game. All we care about is who wins, so we just return the winner. For now, we assume that the pair with the higher number always wins. So the expectation is that the simulation should result in a list of pairs in descending order.

In [8]:
def game(p1, p2):
    return max(p1, p2)

OK, we now have our procedures for working out the pairings, and playing games. For the simulation, we just do this a set number of times.

In [54]:
def swiss(num_pairs, num_rounds):
    if num_pairs % 2:
        raise ValueError("An even number of pairs is required (num_pairs={})".format(num_pairs))
    # if 
    scores = {p:0 for p in range(num_pairs)}
    played = set()
    
    for board in range(num_rounds):
        print("Round {}: Pairings played: {} out of {}".format(board, len(played), num_pairs*(num_pairs-1)//2))
        try:
            games = pair(scores, played)
        except RuntimeError:
            from collections import defaultdict
            d = defaultdict(list)
            for p1, p2 in played:
                d[p1].append(p2)
            for k in sorted(d):
                print(k, "played", list(sorted(d[k])))
            raise
        for p1, p2 in games:
            winner = game(p1, p2)
            scores[winner] += 1
            
    return scores

Counter(swiss(10, 5)).most_common()

Round 0: Pairings played: 0 out of 45
Ranking: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Matching 0... 1, Found 1
Matching 2... 3, Found 3
Matching 4... 5, Found 5
Matching 6... 7, Found 7
Matching 8... 9, Found 9
Round 1: Pairings played: 5 out of 45
Ranking: [0, 2, 4, 6, 8, 1, 3, 5, 7, 9]
Matching 0... 2, Found 2
Matching 4... 6, Found 6
Matching 8... 1, Found 1
Matching 3... 5, Found 5
Matching 7... 9, Found 9
Round 2: Pairings played: 10 out of 45
Ranking: [0, 4, 1, 2, 3, 6, 7, 8, 5, 9]
Matching 0... 4, Found 4
Matching 1... 2, Found 2
Matching 3... 6, Found 6
Matching 7... 8, Found 8
Matching 5... 9, Found 9
Round 3: Pairings played: 15 out of 45
Ranking: [0, 1, 3, 4, 7, 2, 5, 6, 8, 9]
Matching 0... 1, 3, Found 3
Matching 1... 4, Found 4
Matching 7... 2, Found 2
Matching 5... 6, Found 6
Matching 8... 9, 
0 played [1, 2, 3, 4]
1 played [2, 4]
2 played [3]
3 played [5, 6]
4 played [5, 6]
5 played [6, 9]
6 played [7]
7 played [2, 8, 9]
8 played [1, 9]


RuntimeError: Unable to find a game for pair 8